<a href="https://colab.research.google.com/github/ofoofoo/dynamic-token-merging/blob/main/experiments/merge_sequence_test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install ptflops
!pip install fvcore

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename

In [2]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import evaluate
from tqdm import tqdm
from ptflops import get_model_complexity_info

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
import random
import time
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import evaluate
import torch
from fvcore.nn import FlopCountAnalysis

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn").to(device)

ds = load_dataset("abisee/cnn_dailymail", "3.0.0")
test_dataset = ds['test']
subset_size = 100
random_indices = random.sample(range(len(test_dataset)), subset_size)
test_dataset = test_dataset.select(random_indices)
rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [11]:
total_flops = 0
total_time = 0
outputs = []
targets = []

In [4]:
print("Vocab Size", model.config.vocab_size)
print("Hidden Size", model.config.hidden_size)
print(model.lm_head)

Vocab Size 50264
Hidden Size 1024
Linear(in_features=1024, out_features=50264, bias=False)


In [ ]:
'''
        num_pruning_steps = 3
        for i in range(num_pruning_steps):
            for j in range(12 // num_pruning_steps):
                result = model.model.encoder.layers[i * (12 // num_pruning_steps) + j](encoder_hidden_states,
                                                               attention_mask = None,
                                                               layer_head_mask=None,
                                                               output_attentions=True)
                encoder_hidden_states = result[0]
                attention = result[1]
            avg_attention = torch.mean(attention, dim=(1, 2))
            threshold = torch.quantile(avg_attention, 0)
            mask = avg_attention > threshold
            encoder_hidden_states = encoder_hidden_states[mask].unsqueeze(dim=0)
        '''

In [14]:
for example in tqdm(test_dataset):
    article = example['article']
    reference_summary = example['highlights']
    tokens = tokenizer.tokenize(article, max_length=1024, truncation=True)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    token_ids_tensor = torch.tensor(token_ids).unsqueeze(0).to(device)

    with torch.no_grad():
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        model.eval()
        start_time = time.time()
        print('')

        # Embedding as done in the Encoder in
        # https://huggingface.co/transformers/v3.0.2/_modules/transformers/modeling_bart.html#BartModel.forward
        inputs_embeds = model.model.encoder.embed_tokens(token_ids_tensor)
        embed_pos = model.model.encoder.embed_positions(token_ids_tensor)
        encoder_hidden_states = inputs_embeds + embed_pos
        encoder_hidden_states = model.model.encoder.layernorm_embedding(encoder_hidden_states)

        '''
        for i in range(12):
            layer_outputs = model.model.encoder.layers[i](encoder_hidden_states,
                                                   attention_mask = None,
                                                   layer_head_mask=None,
                                                   output_attentions=True)
            encoder_hidden_states = layer_outputs[0]
            attention = layer_outputs[1]
        '''

        num_pruning_steps = 3
        pruning_proportion = 0.2
        for i in range(num_pruning_steps):
            for j in range(12 // num_pruning_steps):
                current_layer_index = i * (12 // num_pruning_steps) + j
                layer_outputs = model.model.encoder.layers[current_layer_index](encoder_hidden_states,
                                                   attention_mask = None,
                                                   layer_head_mask=None,
                                                   output_attentions=True)
                encoder_hidden_states, attention = layer_outputs
            avg_attention = torch.mean(attention, dim=(1, 2))
            threshold = torch.quantile(avg_attention, pruning_proportion)
            mask = avg_attention > threshold
            encoder_hidden_states = encoder_hidden_states[mask].unsqueeze(dim=0)

        # encoder_outputs = model.model.encoder(input_ids = token_ids_tensor, output_attentions=True, output_hidden_states=True)
        # print("Encoder Outputs", encoder_outputs.last_hidden_state.shape)
        # print("Encoder Layer Outputs", encoder_hidden_states.shape)

        decoder_input_ids = torch.tensor([[0, 0]]).to(device)
        for _ in range(128):
            decoder_outputs = model.model.decoder(
                input_ids = decoder_input_ids,
                encoder_hidden_states = encoder_hidden_states
            )
            logits = model.lm_head(decoder_outputs[0])
            predicted_token_id = logits.argmax(dim=-1)[:, -1]
            decoder_input_ids = torch.cat([decoder_input_ids, predicted_token_id.unsqueeze(dim=0)], dim=-1)
            if predicted_token_id == 2:
                break

        generated_summary = tokenizer.decode(decoder_input_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        end_time = time.time()

        real_generated_tokens = model.generate(
            input_ids=token_ids_tensor,
            attention_mask=None
        )
        real_generated_summary = tokenizer.decode(real_generated_tokens[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

        print('')
        print(generated_summary)
        print('----------------')
        print(real_generated_summary)
        print('----------------')
        print(reference_summary)
        total_time += (end_time - start_time)
        outputs.append(generated_summary)
        targets.append(reference_summary)

average_time = total_time / len(test_dataset)

rouge_results = rouge.compute(predictions=outputs, references=targets)
bleu_results = bleu.compute(predictions=outputs, references=targets)

print("Inference Results:")
print(f"Average forward pass time: {average_time:.4f} seconds")
print("ROUGE Scores:", rouge_results)
print("BLEU Score:", bleu_results)

  0%|          | 0/100 [00:00<?, ?it/s]

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



  1%|          | 1/100 [00:06<11:07,  6.74s/it]



----------------
Alleged rape took place on Panama City Beach, Florida, during Spring Break. Hundreds of people watched but did not intervene, authorities claim. Footage was uncovered on a cell phone during an unconnected investigation into a shooting in Troy, Alabama. Delone' Martistee, 22, and Ryan Austin Calhoun, 23, have been suspended from college while they are detained for questioning.
----------------
Police in Troy, Alabama, found the video while investigating a shooting .
Video 'shows two men raping unconscious woman on Florida beach'
Authorities say hundreds of people walk past but don't intervene .
Troy University students Delone' Martistee, 22, and Ryan Austin Calhoun, 23, have been arrested in connection with the alleged incident .
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



  2%|▏         | 2/100 [00:10<08:22,  5.13s/it]

...

 99%|█████████▉| 99/100 [04:59<00:03,  3.04s/it]


Aboriginal teenager who was arrested after anti-terror raids in Melbourne has complained he was victim of police brutality. Cruse, 19, one of five men taken into custody after raids on Saturday morning, said officers assaulted him after handcuffed. His father, Glen, said he was brutally attacked by police during the raids. It comes after the father of one of the arrested teens, said police held a gun and held a gun on his family.
----------------
Eathan Cruse, 19, was one of five men taken into custody after anti-terror raids in Melbourne on Saturday morning. He said officers assaulted him after he had been handcuffed. His father, Glen, also said he was brutally attacked by police during the raid, and his son was hospitalised as a result of injuries he sustained. Victoria Police confirmed a complaint had been made in regards to the treatment of Mr Cruse. The raids were carried out across Melbourne, and reportedly foiled an ANZAC Day terror plot.
----------------
Aboriginal teenager ar

100%|██████████| 100/100 [05:02<00:00,  3.02s/it]


Indian doctors at Kathmandu Hospital say Pakistan sent aid to Nepal. The aid included packets of ready-to-eat meals and food from a hotel. The majority of Nepal's population treats cows as sacred. Eating beef is a religious offence in Nepal.
----------------
Indian doctors at Kathmandu’s Bir Hospital told Mail Today that packets of ‘beef masala’ were sent by Pakistan on Tuesday as part of relief aid to the temblor survivors. The majority-Hindu country treats cows as sacred and there is a blanket ban on slaughtering the animal. The development has the potential to trigger diplomatic acrimony between the South Asian Association for Regional Cooperation member countries.
----------------
Cows are treated as sacred in Nepal, where the majority are Hindus .
Killing a cow used to be punishable by death, but now carries a 12-year jail term .
Nepalese government officials have informed the country's PM .
An internal inquiry is underway and the matter will be raised with Pakistan .


Inference Results:
Average forward pass time: 1.6576 seconds
ROUGE Scores: {'rouge1': 0.33024858946095503, 'rouge2': 0.13350941561362567, 'rougeL': 0.22530669726234168, 'rougeLsum': 0.2786344300013116}
BLEU Score: {'bleu': 0.09898970818268119, 'precisions': [0.4242881794650561, 0.1667033208709039, 0.08411843876177658, 0.044632639047837035], 'brevity_penalty': 0.7754482235457536, 'length_ratio': 0.7972484952708513, 'translation_length': 4636, 'reference_length': 5815}


In [ ]:
for example in tqdm(test_dataset):
    article = example['article']
    reference_summary = example['highlights']
    tokens = tokenizer.tokenize(article, max_length=1024, truncation=True)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    token_ids_tensor = torch.tensor(token_ids).to(device)

    with torch.no_grad():
        token_embeddings = model.model.shared(token_ids_tensor)

    importance_scores = torch.norm(token_embeddings, dim=1)

    threshold = torch.quantile(importance_scores, 0.25)
    mask = importance_scores > threshold
    pruned_token_ids = token_ids_tensor[mask]

    if pruned_token_ids.numel() == 0:
        pruned_token_ids = token_ids_tensor  # Fall back to original tokens

    inputs = {
        'input_ids': pruned_token_ids.unsqueeze(0).to(device),
        'attention_mask': torch.ones_like(pruned_token_ids).unsqueeze(0).to(device)
    }
    print("attention mask shape", inputs['attention_mask'].shape)

    # flops_analysis = FlopCountAnalysis(model, inputs['input_ids'])
    # total_flops += flops_analysis.total()

    start_time = time.time()

    with torch.no_grad():
        encoder_hidden_states = token_embeddings.unsqueeze(dim=0)
        for i in range(4):
            for i in range(3):
                result = model.model.encoder.layers[i](encoder_hidden_states, attention_mask = None, layer_head_mask=None, output_attentions=True)
                encoder_hidden_states = result[0]
                attention = result[1]
            avg_attention = torch.mean(attention, dim=(1, 2))
            threshold = torch.quantile(avg_attention, 0.2)
            mask = avg_attention > threshold
            encoder_hidden_states = encoder_hidden_states[mask].unsqueeze(dim=0)
        print(encoder_hidden_states.shape)

        decode_ids = tokenizer.encode("<s>", return_tensors="pt")
        for _ in range(10):
            print(decode_ids.shape)
            seq_length = decode_ids.shape[1]
            # causal_mask = torch.tril(torch.ones((seq_length, seq_length), device=device)).unsqueeze(0)
            # causal_mask = causal_mask.masked_fill(causal_mask == 0, float('-inf')).masked_fill(causal_mask == 1, 0.0)
            decoder_outputs = model.model.decoder(input_ids = decode_ids, encoder_hidden_states = encoder_hidden_states)
            logits = model.lm_head(decoder_outputs[0])
            next_token_id = torch.argmax(logits, dim=-1)
            print(logits, logits.shape)
            print(next_token_id)
            decode_ids = torch.cat([decode_ids, next_token_id], dim=-1)
        print(decode_ids)
        # causal_mask = torch.tril(torch.ones((seq_length, seq_length), device=device)).unsqueeze(0)
        #result = model.model.decoder(encoder_hidden_states)
        #print(result)
    assert False

    summary_ids = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask']
    )

    end_time = time.time()
    print(summary_ids, summary_ids.shape)

    generated_summary = tokenizer.decode(
        summary_ids[0],
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )
    print(generated_summary)

    assert False
    total_time += (end_time - start_time)
    outputs.append(generated_summary)
    targets.append(reference_summary)

average_flops = total_flops / len(test_dataset)
average_time = total_time / len(test_dataset)

rouge_results = rouge.compute(predictions=outputs, references=targets)
bleu_results = bleu.compute(predictions=outputs, references=targets)

print("Inference Results:")
print(f"Average FLOPs: {average_flops:.2e}")
print(f"Average forward pass time: {average_time:.4f} seconds")
print("ROUGE Scores:", rouge_results)
print("BLEU Score:", bleu_results)


  0%|          | 0/100 [00:00<?, ?it/s]

attention mask shape torch.Size([1, 751])
torch.Size([1, 416, 1024])
torch.Size([1, 3])
tensor([[[11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290],
         [11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290],
         [11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290]]]) torch.Size([1, 3, 50264])
tensor([[0, 0, 0]])
torch.Size([1, 6])
tensor([[[11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290],
         [11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290],
         [11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290],
         [11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290],
         [11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290],
         [11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290]]]) torch.Size([1, 6, 50264])
tensor([[0, 0, 0, 0, 0, 0]])
torch.Size([1, 12])
tensor([[[11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.0885,  0.6290],
         [11.9234,  0.2804,  1.0227,  ...,  0.8962, -0.088

  0%|          | 0/100 [00:12<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import torch.nn as nn
from fvcore.nn import FlopCountAnalysis
from transformers.modeling_outputs import BaseModelOutput

class PrunedEncoder(nn.Module):
    def __init__(self, encoder, num_pruning_steps=3, pruning_proportion=0):
        super(PrunedEncoder, self).__init__()
        self.encoder = encoder
        self.num_pruning_steps = num_pruning_steps
        self.pruning_proportion = pruning_proportion

    def forward(self, input_ids):
        inputs_embeds = self.encoder.embed_tokens(input_ids)
        embed_pos = self.encoder.embed_positions(input_ids)
        encoder_hidden_states = inputs_embeds + embed_pos
        encoder_hidden_states = self.encoder.layernorm_embedding(encoder_hidden_states)

        attention = None
        for i in range(self.num_pruning_steps):
            for j in range(12 // self.num_pruning_steps):
                current_layer_index = i * (12 // self.num_pruning_steps) + j
                layer_outputs = self.encoder.layers[current_layer_index](
                    encoder_hidden_states,
                    attention_mask=None,
                    layer_head_mask=None,
                    output_attentions=True
                )
                encoder_hidden_states, attention = layer_outputs[:2]
            avg_attention = torch.mean(attention, dim=(1, 2))
            threshold = torch.quantile(avg_attention, self.pruning_proportion)
            mask = avg_attention > threshold
            mask = mask.squeeze(0) # [seq_len]
            #encoder_hidden_states[mask].unsqueeze(dim=0)
            encoder_hidden_states = encoder_hidden_states[:, mask, :]

        return encoder_hidden_states



class PrunedModel(nn.Module):
    def __init__(self, model, num_pruning_steps=3, pruning_proportion=0):
        super(PrunedModel, self).__init__()
        self.encoder = PrunedEncoder(model.model.encoder, num_pruning_steps, pruning_proportion)
        self.decoder = model.model.decoder
        self.lm_head = model.lm_head
        self.config = model.config

    def forward(self, input_ids, decoder_input_ids):
        encoder_outputs = self.encoder(input_ids)
        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            encoder_hidden_states=encoder_outputs,
            encoder_attention_mask=None,
            past_key_values=None,
            use_cache=False,
            output_attentions=False,
            output_hidden_states=False,
            return_dict=True,
        )
        sequence_output = decoder_outputs.last_hidden_state
        logits = self.lm_head(sequence_output)
        return logits


class ReferenceModel(nn.Module):
    def __init__(self, model):
        super(ReferenceModel, self).__init__()
        self.model = model

    def forward(self, input_ids, decoder_input_ids):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=None,
            decoder_input_ids=decoder_input_ids,
            use_cache=False,
            output_attentions=False,
            output_hidden_states=False,
            return_dict=True,
        )
        return outputs.logits

all_results = {}

for i in [0.2]:
  print(f"RUNNING WITH PRUNING PROPORTION: {i}")
  total_flops_pruned = 0
  total_flops_ref = 0
  total_time = 0
  total_time_ref = 0
  outputs = []
  outputs_ref = []
  targets = []

  pruned_model_instance = PrunedModel(model, pruning_proportion=i).to(device)
  reference_model_instance = ReferenceModel(model).to(device)

  for example in tqdm(test_dataset):
      article = example['article']
      reference_summary = example['highlights']
      tokens = tokenizer.tokenize(article, max_length=1024, truncation=True)
      token_ids = tokenizer.convert_tokens_to_ids(tokens)
      token_ids_tensor = torch.tensor([token_ids]).to(device)  # [1, seq_len]

      decoder_input_ids = torch.tensor([[model.config.decoder_start_token_id]]).to(device)

      with torch.no_grad():
          print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
          model.eval()
          print('')

          # measure FLOPs: need to run this before timing
          flops_pruned = FlopCountAnalysis(
              pruned_model_instance,
              (token_ids_tensor, decoder_input_ids)
          ).total()
          total_flops_pruned += flops_pruned

          # pruned model generation
          start_time = time.time()
          generated_tokens = model.generate(
              encoder_outputs=BaseModelOutput(
                  last_hidden_state=pruned_model_instance.encoder(token_ids_tensor)
              ),
              decoder_input_ids=decoder_input_ids,
              max_length=128,
              num_beams=4,
              early_stopping=True
          )
          generated_summary = tokenizer.decode(
              generated_tokens[0],
              skip_special_tokens=True,
              clean_up_tokenization_spaces=True
          )
          end_time = time.time()
          total_time += (end_time - start_time)

          flops_ref = FlopCountAnalysis(
              reference_model_instance,
              (token_ids_tensor, decoder_input_ids)
          ).total()
          total_flops_ref += flops_ref

          # reference model generation
          start_time_ref = time.time()
          real_generated_tokens = model.generate(
              input_ids=token_ids_tensor,
              attention_mask=None,
              max_length=128,
              num_beams=4,
              early_stopping=True
          )
          real_generated_summary = tokenizer.decode(
              real_generated_tokens[0],
              skip_special_tokens=True,
              clean_up_tokenization_spaces=True
          )
          end_time_ref = time.time()
          total_time_ref += (end_time_ref - start_time_ref)

          print('')
          print(generated_summary)
          print('----------------')
          print(real_generated_summary)
          print('----------------')
          print(reference_summary)
          outputs.append(generated_summary)
          outputs_ref.append(real_generated_summary)
          targets.append(reference_summary)

  average_flops_pruned = total_flops_pruned / len(test_dataset)
  average_flops_ref = total_flops_ref / len(test_dataset)
  average_time = total_time / len(test_dataset)
  average_time_ref = total_time_ref / len(test_dataset)

  rouge_results = rouge.compute(predictions=outputs, references=targets)
  bleu_results = bleu.compute(predictions=outputs, references=targets)

  rouge_results_ref = rouge.compute(predictions=outputs_ref, references=targets)
  bleu_results_ref = bleu.compute(predictions=outputs_ref, references=targets)

  print("Inference Results:")
  print(f"Average forward pass time: {average_time:.4f} seconds")
  print(f"Average forward pass time (reference): {average_time_ref:.4f} seconds")
  print(f"Average FLOPs for pruned model: {average_flops_pruned:.2f}")
  print(f"Average FLOPs for reference model: {average_flops_ref:.2f}")
  print("ROUGE Scores:", rouge_results)
  print("ROUGE Scores (reference):", rouge_results_ref)
  print("BLEU Score:", bleu_results)
  print("BLEU Score (reference):", bleu_results_ref)

  summary_stats = {"average_flops_pruned": average_flops_pruned,
      "average_flops_ref": average_flops_ref,
      "average_time_pruned": average_time,
      "average_time_ref": average_time_ref,
      "average_rouge_pruned": rouge_results,
      "average_rouge_ref": rouge_results_ref,
      "average_bleu_pruned": bleu_results,
      "average_bleu_ref": bleu_results_ref}
  all_results[i] = summary_stats